In [9]:
pip install pandas numpy scikit-learn cryptography

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from cryptography.fernet import Fernet
import pickle
import json
import base64  # Import base64 for encoding/decoding bytes

# Data encryption class for HIPAA compliance simulation
class DataSecurity:
    def __init__(self):
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

    def encrypt_data(self, data):
        # Convert numpy array to list and then to JSON string before encryption
        if isinstance(data, np.ndarray):
            data = data.tolist()
        # Encode the pickled data to base64 before encryption
        pickled_data = pickle.dumps(data)
        encoded_data = base64.b64encode(pickled_data).decode('utf-8')  # Encode to base64 string
        encrypted_data = self.cipher.encrypt(encoded_data.encode())
        return encrypted_data

    def decrypt_data(self, encrypted_data):
        # Decrypt and convert back from JSON string
        decrypted_data = self.cipher.decrypt(encrypted_data).decode()
        decoded_data = base64.b64decode(decrypted_data.encode('utf-8'))  # Decode from base64 string
        data = pickle.loads(decoded_data)
        return data

# ... (Rest of the code remains the same) ...

# Load and preprocess the data
def load_and_preprocess_data():
    df = pd.read_csv('/content/Heart.csv')

    chest_pain_map = {'typical': 1, 'asymptomatic': 2, 'nonanginal': 3, 'nontypical': 4}
    thal_map = {'fixed': 1, 'normal': 2, 'reversable': 3, 'NA': 0}
    ahd_map = {'No': 0, 'Yes': 1}

    df['ChestPain'] = df['ChestPain'].map(chest_pain_map)
    df['Thal'] = df['Thal'].map(thal_map)
    df['AHD'] = df['AHD'].map(ahd_map)

    df = df.replace('NA', np.nan)
    df = df.dropna()

    X = df.drop(['AHD'], axis=1)
    y = df['AHD']

    return X, y

# Main prediction class
class HeartDiseasePredictor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = LogisticRegression(max_iter=1000)
        self.security = DataSecurity()

    def train_model(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        self.model.fit(X_train_scaled, y_train)

        y_pred = self.model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"Model Accuracy: {accuracy:.2%}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        return accuracy

    def predict(self, patient_data):
        scaled_data = self.scaler.transform(patient_data)
        prediction = self.model.predict(scaled_data)
        probability = self.model.predict_proba(scaled_data)

        return prediction, probability

    def save_model(self, filename='heart_disease_model.pkl'):
        model_data = {
            'scaler': self.scaler,
            'model': self.model
        }
        with open(filename, 'wb') as f:
            encrypted_data = self.security.encrypt_data(pickle.dumps(model_data))
            f.write(encrypted_data)

    def load_model(self, filename='heart_disease_model.pkl'):
        with open(filename, 'rb') as f:
            encrypted_data = f.read()
            decrypted_data = self.security.decrypt_data(encrypted_data)
            model_data = pickle.loads(decrypted_data.encode())

            self.scaler = model_data['scaler']
            self.model = model_data['model']

# Main execution
def main():
    X, y = load_and_preprocess_data()

    predictor = HeartDiseasePredictor()
    accuracy = predictor.train_model(X, y)

    predictor.save_model()

    # Example prediction for a new patient
    sample_patient = X.iloc[0].values.reshape(1, -1)
    encrypted_patient_data = predictor.security.encrypt_data(sample_patient)

    # Decrypt and predict
    decrypted_patient_data = predictor.security.decrypt_data(encrypted_patient_data)
    patient_array = np.array(decrypted_patient_data).reshape(1, -1)

    prediction, probability = predictor.predict(patient_array)

    print("\nSample Patient Prediction:")
    print(f"Prediction: {'Heart Disease' if prediction[0] == 1 else 'No Heart Disease'}")
    print(f"Probability: {probability[0][1]:.2%}")

if __name__ == "__main__":
    main()

Model Accuracy: 83.33%

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86        36
           1       0.82      0.75      0.78        24

    accuracy                           0.83        60
   macro avg       0.83      0.82      0.82        60
weighted avg       0.83      0.83      0.83        60


Sample Patient Prediction:
Prediction: No Heart Disease
Probability: 13.79%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
